本节我们将了解一种新的任务——分类（Classification）。

![](./Images/6/1.png)

与[回归](./3.回归.md)不同，分类任务输出的是类别。比如：

![](./Images/6/2.png)

但我们暂时不研究上述例子，而是讨论一下宝可梦系别分类的问题。

![](./Images/6/3.png)

当知道了一个宝可梦的一些特征值后，就可以预测它属于什么系别。

![](./Images/6/4.png)

或许你会问这有什么用呢？当然有用！根据特征值预测系别后，就可以参考[属性相克表](http://wiki.52poke.com/wiki/%E5%B1%9E%E6%80%A7%E7%9B%B8%E5%85%8B%E8%A1%A8)进行战斗啦！

![](./Images/6/5.png)

## How to do Classification

当我们已经有了训练数据，该怎么做分类任务呢？

![](./Images/6/6.png)

考虑使用之前的线性模型，以二分类任务为例，如果类别是1，让对应的$\hat{y}$值为1；如果类别是2，让对应的$\hat{y}$值为-1。

当得到训练好的模型后，如果输出值接近1，那么该样本的类别就是1；如果输出值接近-1，那么该样本的类别就是2。

![](./Images/6/7.png)

但如果有些样本的输出值远大于1时，如上图右边所示。这会造成损失值非常大，训练时为了让误差函数的值减小。

线性模型会逐渐偏向这些样本，最终得到如上图所示紫线所示的模型。这显然不是我们想要的结果，我们希望得到

绿色的模型。于是，我们可能并不能直接简单地使用上述线性模型。

在开始新模型之前，让我们先看一个例子：

![](./Images/6/8.png)

假设有Box 1和Box 2两个箱子，里面分别有蓝色和绿色两种颜色的球，从Box 1和Box 2取球的概率分别是$\frac{2}{3}$和

$\frac{1}{3}$，其他概率如上图所示。问题来了，如果我们已经取到了一个蓝色的球，那么它来自Box 1的概率是多少？

使用贝叶斯定理，我们容易得到：$$P\left(B_1 | Blue \right) = \frac{\frac{4}{5} * \frac{2}{3}}{\frac{4}{5} * \frac{2}{3} + \frac{2}{5} * \frac{1}{3}} = 0.8$$

对于分类任务，它对应下图：

![](./Images/6/9.png)

箱子相当于类别（Box 1对应Class 1），x相当于样本的属性值（或者说样本），我们的目标是求到给定x时，

对应类别的概率。比如，$P \left( C_1 | x \right) = 0.4$， $P \left( C_2 | x \right) = 0.6$，$P \left( C_1 | x \right) < P \left( C_2 | x \right)$，可以认为该样本的类别为$C_2$。

回到宝可梦的例子，只考虑两种系别Water和Normal。

![](./Images/6/10.png)

统计了训练集中样本后，我们得到$P \left( C_1 \right) = 0.56$，$P \left( C_2 \right) = 0.44$。

接下来就是要求到：

![](./Images/6/11.png)

这里我们暂时就考虑两个宝可梦的两个属性SP Defense和Defense，

![](./Images/6/12.png)

并假设类中的样本服从[高斯分布](https://www.wikiwand.com/en/Normal_distribution)。

![](./Images/6/13.png)

用什么方法找到合适的$\mu$和$\Sigma$呢？答案是[最大似然估计](https://www.wikiwand.com/en/Maximum_likelihood_estimation)。

![](./Images/6/14.png)

具体过程如下：

![](./Images/6/15.png)

通过上述公式，我们可以计算得到Water和Normal两类对应的$\mu$和$\Sigma$值。

![](./Images/6/16.png)

终于，我们可以进行分类了

![](./Images/6/17.png)

示例如下：

In [1]:
from scipy.stats import multivariate_normal

In [2]:
import numpy as np

In [3]:
u1 = np.array([75.0, 71.3])
u2 = np.array([55.6, 59.8])
sigma1 = np.array([[874, 327], [327, 929]])
sigma2 = np.array([[847, 422], [422, 685]])

In [4]:
g1 = multivariate_normal(u1, sigma1)

In [5]:
g2 = multivariate_normal(u2, sigma2)

In [6]:
x = np.array([76.2, 58.4])

In [7]:
c1 = 0.56
c2 = 0.44

In [8]:
c1 * g1.pdf(x) / (c1 * g1.pdf(x) + c2 * g2.pdf(x))

0.5604330716936666

In [9]:
c2 * g2.pdf(x) / (c1 * g1.pdf(x) + c2 * g2.pdf(x))

0.4395669283063334

[scipy.stats.multivariate_normal官方文档](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html)。

所以，Defence为76.2，SP Defence为58.4的一个宝可梦的预测系别是Water。

测试集上的结果如下：

![](./Images/6/18.png)


结果不太好呀！对于不同的Class我们可以得到其对应的$\mu$和$\Sigma$，每个$\Sigma$的参数量和特征向量的平方成正比，

也就是说$\Sigma$会使得模型的参数量增长地很快，参数量一大，就容易造成过拟合现象。

一般的做法使共享同一个$\Sigma$。

![](./Images/6/19.png)

## Modifying Model

于是我们将模型改成如下形式：

![](./Images/6/20.png)

$\mu$的计算和之前一样，$\Sigma$的计算有点不同。那么它的效果怎么样呢？

![](./Images/6/21.png)

和之前比较的话，着实提高不少。

In [10]:
sigma = 79 / 140 * sigma1 + 61 / 140 * sigma2

In [11]:
g1 = multivariate_normal(u1, sigma)

In [12]:
g2 = multivariate_normal(u2, sigma)

In [13]:
c1 * g1.pdf(x) / (c1 * g1.pdf(x) + c2 * g2.pdf(x))

0.6057343451659792

In [14]:
c2 * g2.pdf(x) / (c1 * g1.pdf(x) + c2 * g2.pdf(x))

0.3942656548340208

我们在[引言](1.引言.md)中提到机器学习分成三步，这里照样是三步：

![](./Images/6/22.png)

这里可能会有疑问，为什么一定要选高斯分布呢？答案是不一定，我们也可以做别的假设。

![](./Images/6/23.png)

## Posterior Probability

将$P\left( C_1 | x \right)$整理一下，可以得到：

![](./Images/6/24.png)

$z$究竟是什么？我们可以进一步整理，最后得到：

![](./Images/6/25.png)

在生成模型中，我们做的是用某些方法找出$N_1$，$N_2$，$\mu^1$，$\mu^2$，$\Sigma$的值，用以计算$w$和$b$的值，

然后代入$\sigma \left( w * x + b\right)$中计算概率值。既然我们最终的目的是得到$w$和$b$的值，用得着这么麻烦计算

$N_1$，$N_2$，$\mu^1$，$\mu^2$，$\Sigma$的值吗？能不能直接求到呢？

尤其是我们在回归一节见识到了如何使用梯度下降法计算线性模型$w$和$b$的值。

这便是下一节的内容。

我们用[Kaggle](https://www.kaggle.com/)上的[Pokemon with stats](https://www.kaggle.com/abcsds/pokemon)数据集做个示例。

In [15]:
import numpy as np
import pandas as pd

In [16]:
data_path = './Codes/6/Pokemon.csv'

In [17]:
data = pd.read_csv(data_path, index_col='#')

In [18]:
data

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 1 to 721
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        800 non-null    object
 1   Type 1      800 non-null    object
 2   Type 2      414 non-null    object
 3   Total       800 non-null    int64 
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 75.8+ KB


In [20]:
train_data = data[data.index < 400]

In [21]:
train_data['Type 1'].value_counts()

Water       79
Normal      61
Grass       38
Bug         38
Fire        33
Psychic     30
Rock        23
Electric    21
Ground      18
Poison      18
Steel       15
Dragon      15
Fighting    14
Ice         13
Dark        12
Ghost       10
Fairy        7
Name: Type 1, dtype: int64

In [22]:
water_count = 79
normal_count = 61

In [43]:
c1 = water_count / (water_count + normal_count)
c2 = normal_count / (water_count + normal_count)

In [24]:
train_data

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
395,Empoleon,Water,Steel,530,84,86,88,111,101,60,4,False
396,Starly,Normal,Flying,245,40,55,30,30,30,60,4,False
397,Staravia,Normal,Flying,340,55,75,50,40,40,80,4,False


In [25]:
train_data[train_data['Type 1'] == 'Water'][['Defense', 'Sp. Def']]

,Defense,Sp. Def
#,,
7,65,64
8,80,80
9,100,105
9,120,115
54,48,50
...,...,...
382,90,140
382,90,160
393,53,56


In [26]:
train_data[train_data['Type 1'] == 'Normal'][['Defense', 'Sp. Def']]

,Defense,Sp. Def
#,,
16,40,35
17,55,50
18,75,70
18,80,80
19,35,35
...,...,...
352,70,120
396,30,30
397,50,40


In [27]:
water_defense_sp_denfense_data = train_data[train_data['Type 1'] == 'Water'][['Defense', 'Sp. Def']]

In [28]:
normal_defense_sp_denfense_data = train_data[train_data['Type 1'] == 'Normal'][['Defense', 'Sp. Def']]

In [29]:
np.mean(water_defense_sp_denfense_data)

Defense    75.037975
Sp. Def    71.329114
dtype: float64

In [30]:
np.mean(normal_defense_sp_denfense_data)

Defense    55.557377
Sp. Def    59.836066
dtype: float64

In [31]:
water_mean = np.mean(water_defense_sp_denfense_data)

In [32]:
normal_mean = np.mean(normal_defense_sp_denfense_data)

In [33]:
water_mean, normal_mean

(Defense    75.037975
 Sp. Def    71.329114
 dtype: float64,
 Defense    55.557377
 Sp. Def    59.836066
 dtype: float64)

使用以下公式进行计算：

![](./Images/6/26.png)

In [34]:
water_defense_sp_denfense_data.shape, water_mean.shape

((79, 2), (2,))

In [35]:
def calculate_sigma(data):
    result = []
    for item in data:
        item = item.reshape((1, -1))
        result.append(item.T.dot(item))
    return np.sum(result, axis=0) / len(data)

In [36]:
water_sigma = calculate_sigma((water_defense_sp_denfense_data - water_mean).values)

In [37]:
normal_sigma = calculate_sigma((normal_defense_sp_denfense_data - normal_mean).values)

In [38]:
water_sigma

array([[873.85931742, 327.20269188],
       [327.20269188, 928.67649415]])

In [39]:
normal_sigma

array([[468.27949476, 197.76350443],
       [197.76350443, 552.69443698]])

In [45]:
c1, c2

(0.5642857142857143, 0.4357142857142857)

In [46]:
g1 = multivariate_normal(water_mean, water_sigma)
g2 = multivariate_normal(normal_mean, normal_sigma)
x = np.array([76.2, 58.4])

In [47]:
c1 * g1.pdf(x) / (c1 * g1.pdf(x) + c2 * g2.pdf(x))

0.5322810808752668

In [48]:
c2 * g2.pdf(x) / (c1 * g1.pdf(x) + c2 * g2.pdf(x))

0.4677189191247331

## 推荐阅读

[Generative model](https://www.wikiwand.com/en/Generative_model)

[Machine Learning: Generative and Discriminative Models](https://cedar.buffalo.edu/~srihari/CSE574/Discriminative-Generative.pdf)